Achintya Yedavalli

# Assignment 2: Exploring Pre-Processing Techniques

Welcome to TweetMiner, the leading organization in Twitter data analysis! As an NLP scientist in our team, you're entrusted with the task of extracting the most relevant tweets based on input hashtags. For instance, if the hashtag is "#abortion," we expect you to extract the top N (let's say N=10) tweets that truly discuss the topic of "abortion." Similarly, for a hashtag like "#politicaladvertising," your algorithm should identify and extract the top N (again, let's use N=10) tweets about "political advertising".

Despite the availability of advanced algorithms, we're interested in exploring a few fundamental approaches as given below.


In [2]:
# load the dataset!!

de_file = open("australian_election_2019_tweets.txt", "r")

list_sentences = de_file.read().replace('\n', ' ').split(".")

list_sentences

['After the climate election: shellshocked green groups remain resolute https://t',
 'co/wyJzmAcyiD @narendramodi @smritiirani Coverage of indian election on SBS tv channel, Australia',
 ' Jai hind 🇮🇳🙏 https://t',
 'co/90qplBEAf8 @workmanalice Do you know if Facebook is releasing an election post-mortem in Australia? They looked into the midterms, but were we important enough to bother? @vanbadham We all understand we have a compulsory preference system',
 ' Vote 1 mightn’t go to the major but 2 or 3 usually does',
 ' Majority of Australia wanted LNP, that’s the facts',
 ' This is nothing like the USA System',
 ' Shares were mixed in Asia, with India and Australia leading gains for the region following elections that looked set to keep incumbents in office',
 ' https://t',
 "co/krRhPYuRID Australia's pollsters to review incorrect election forecasts https://t",
 'co/isy2SPg7L5 It is disappointing that @tanya_plibersek has ruled herself out of the @AustralianLabor leadership challenge - 

## 1. Simple Word-Overlap Based Match (Search V1.0)

We're going to implement the following:

### a. Offline Processing

- load tweets from file

- remove duplicate tweets

- remove URLs, mentions, hashtags, & non-english text